# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [6]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 11ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [9]:
# Train the model
normalized_input_model.fit(X_train,
                          y_train,
                          batch_size=32,
                          epochs=150,
                          validation_data = (X_val,y_val))

Epoch 1/150
33/33 [==============================] - 1s 9ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train,
                   y_train_scaled,
                   batch_size=32,
                   epochs=150,
                   validation_data = (X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 10ms/step - loss: 0.4849 - mse: 0.4849 - val_loss: 0.1854 - val_mse: 0.1854
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2484 - mse: 0.2484 - val_loss: 0.1457 - val_mse: 0.1457
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.1288 - val_mse: 0.1288
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1772 - mse: 0.1772 - val_loss: 0.1190 - val_mse: 0.1190
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1511 - mse: 0.1511 - val_loss: 0.1213 - val_mse: 0.1213
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1403 - mse: 0.1403 - val_loss: 0.1244 - val_mse: 0.1244
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1217 - mse: 0.1217 - val_loss: 0.1282 - val_mse: 0.1282
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1141 - 

33/33 [==============================] - 0s 4ms/step - loss: 0.0202 - mse: 0.0202 - val_loss: 0.1390 - val_mse: 0.1390
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0199 - mse: 0.0199 - val_loss: 0.1373 - val_mse: 0.1373
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0197 - mse: 0.0197 - val_loss: 0.1406 - val_mse: 0.1406
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0199 - mse: 0.0199 - val_loss: 0.1384 - val_mse: 0.1384
Epoch 67/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0192 - mse: 0.0192 - val_loss: 0.1379 - val_mse: 0.1379
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0189 - mse: 0.0189 - val_loss: 0.1394 - val_mse: 0.1394
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0183 - mse: 0.0183 - val_loss: 0.1410 - val_mse: 0.1410
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0178 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.1390 - val_mse: 0.1390
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0085 - mse: 0.0085 - val_loss: 0.1394 - val_mse: 0.1394
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1390 - val_mse: 0.1390
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0083 - mse: 0.0083 - val_loss: 0.1383 - val_mse: 0.1383
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.1387 - val_mse: 0.1387
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0082 - mse: 0.0082 - val_loss: 0.1395 - val_mse: 0.1395
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0080 - mse: 0.0080 - val_loss: 0.1390 - val_mse: 0.1390
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [12]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0062 - mse: 0.0062


[0.006153286900371313, 0.006153286900371313]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [13]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1379 - mse: 0.1379


[0.13790233433246613, 0.13790233433246613]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [14]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

9/9 [==============================] - 0s 2ms/step


29182.150735258474

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [15]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer = 'he_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(loss='mse',
                optimizer='SGD',
                metrics = ['mse'])
# Train the model
he_model.fit(X_train,
            y_train_scaled,
            batch_size=32,
            epochs=150,
            validation_data = (X_val, y_val_scaled))


Epoch 1/150
33/33 [==============================] - 1s 9ms/step - loss: 0.5131 - mse: 0.5131 - val_loss: 0.2265 - val_mse: 0.2265
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.2363 - mse: 0.2363 - val_loss: 0.1817 - val_mse: 0.1817
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1990 - mse: 0.1990 - val_loss: 0.1657 - val_mse: 0.1657
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1719 - mse: 0.1719 - val_loss: 0.1568 - val_mse: 0.1568
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1562 - mse: 0.1562 - val_loss: 0.1494 - val_mse: 0.1494
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1456 - mse: 0.1456 - val_loss: 0.1530 - val_mse: 0.1530
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1365 - mse: 0.1365 - val_loss: 0.1458 - val_mse: 0.1458
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1233 - m

33/33 [==============================] - 0s 4ms/step - loss: 0.0263 - mse: 0.0263 - val_loss: 0.1217 - val_mse: 0.1217
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0256 - mse: 0.0256 - val_loss: 0.1222 - val_mse: 0.1222
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0255 - mse: 0.0255 - val_loss: 0.1212 - val_mse: 0.1212
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1211 - val_mse: 0.1211
Epoch 67/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1208 - val_mse: 0.1208
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1211 - val_mse: 0.1211
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0242 - mse: 0.0242 - val_loss: 0.1211 - val_mse: 0.1211
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0242 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.1213 - val_mse: 0.1213
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.1206 - val_mse: 0.1206
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.1215 - val_mse: 0.1215
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1209 - val_mse: 0.1209
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1213 - val_mse: 0.1213
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on training data
he_model.evaluate(X_train,y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0093 - mse: 0.0093


[0.009279013611376286, 0.009279013611376286]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [17]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1215 - mse: 0.1215


[0.12154987454414368, 0.12154987454414368]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [18]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(loss='mse',
                   optimizer='SGD',
                   metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 9ms/step - loss: 0.4754 - mse: 0.4754 - val_loss: 0.2368 - val_mse: 0.2368
Epoch 2/150
33/33 [==============================] - 0s 5ms/step - loss: 0.2516 - mse: 0.2516 - val_loss: 0.1738 - val_mse: 0.1738
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1953 - mse: 0.1953 - val_loss: 0.1464 - val_mse: 0.1464
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1688 - mse: 0.1688 - val_loss: 0.1265 - val_mse: 0.1265
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1494 - mse: 0.1494 - val_loss: 0.1175 - val_mse: 0.1175
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1372 - mse: 0.1372 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1311 - mse: 0.1311 - val_loss: 0.1082 - val_mse: 0.1082
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1209 - m

33/33 [==============================] - 0s 4ms/step - loss: 0.0235 - mse: 0.0235 - val_loss: 0.1054 - val_mse: 0.1054
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1057 - val_mse: 0.1057
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1045 - val_mse: 0.1045
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0223 - mse: 0.0223 - val_loss: 0.1042 - val_mse: 0.1042
Epoch 67/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1051 - val_mse: 0.1051
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1058 - val_mse: 0.1058
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1073 - val_mse: 0.1073
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0206 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1096 - val_mse: 0.1096
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1113 - val_mse: 0.1113
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1100 - val_mse: 0.1100
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1105 - val_mse: 0.1105
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0076 - mse: 0.0076


[0.0075878421775996685, 0.0075878421775996685]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1102 - mse: 0.1102


[0.11015395820140839, 0.11015395820140839]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [21]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
rmsprop_model.compile(loss='mse',
                     optimizer='rmsprop',
                     metrics=['mse'])
# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 9ms/step - loss: 0.2867 - mse: 0.2867 - val_loss: 0.1531 - val_mse: 0.1531
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1597 - mse: 0.1597 - val_loss: 0.0981 - val_mse: 0.0981
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1187 - mse: 0.1187 - val_loss: 0.0979 - val_mse: 0.0979
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0954 - mse: 0.0954 - val_loss: 0.1253 - val_mse: 0.1253
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0724 - mse: 0.0724 - val_loss: 0.1125 - val_mse: 0.1125
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0595 - mse: 0.0595 - val_loss: 0.1243 - val_mse: 0.1243
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0524 - mse: 0.0524 - val_loss: 0.1213 - val_mse: 0.1213
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0432 - m

33/33 [==============================] - 0s 4ms/step - loss: 0.0082 - mse: 0.0082 - val_loss: 0.1094 - val_mse: 0.1094
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.1078 - val_mse: 0.1078
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.0990 - val_mse: 0.0990
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.1005 - val_mse: 0.1005
Epoch 67/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0074 - mse: 0.0074 - val_loss: 0.1129 - val_mse: 0.1129
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1049 - val_mse: 0.1049
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0928 - val_mse: 0.0928
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0085 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0945 - val_mse: 0.0945
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.1048 - val_mse: 0.1048
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.0839 - val_mse: 0.0839
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.1012 - val_mse: 0.1012
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.0942 - val_mse: 0.0942
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0032 - mse: 0.0032 - val_loss: 0.1008 - val_mse: 0.1008
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.0905 - val_mse: 0.0905
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train,y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0019 - mse: 0.0019   


[0.0019304396118968725, 0.0019304396118968725]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [23]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val,y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.0946 - mse: 0.0946


[0.09457096457481384, 0.09457096457481384]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [24]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(loss='mse',
                  optimizer='Adam',
                  metrics=['mse'])
# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 13ms/step - loss: 0.3174 - mse: 0.3174 - val_loss: 0.1273 - val_mse: 0.1273
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1330 - mse: 0.1330 - val_loss: 0.1338 - val_mse: 0.1338
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0904 - mse: 0.0904 - val_loss: 0.1189 - val_mse: 0.1189
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0656 - mse: 0.0656 - val_loss: 0.1421 - val_mse: 0.1421
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0502 - mse: 0.0502 - val_loss: 0.1396 - val_mse: 0.1396
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0380 - mse: 0.0380 - val_loss: 0.1334 - val_mse: 0.1334
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0302 - mse: 0.0302 - val_loss: 0.1309 - val_mse: 0.1309
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0261 - 

33/33 [==============================] - 0s 4ms/step - loss: 5.1997e-04 - mse: 5.1997e-04 - val_loss: 0.1091 - val_mse: 0.1091
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 7.8277e-04 - mse: 7.8277e-04 - val_loss: 0.1172 - val_mse: 0.1172
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 7.4952e-04 - mse: 7.4952e-04 - val_loss: 0.1146 - val_mse: 0.1146
Epoch 66/150
33/33 [==============================] - 0s 4ms/step - loss: 7.0270e-04 - mse: 7.0270e-04 - val_loss: 0.1104 - val_mse: 0.1104
Epoch 67/150
33/33 [==============================] - 0s 4ms/step - loss: 5.3321e-04 - mse: 5.3321e-04 - val_loss: 0.1192 - val_mse: 0.1192
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 7.7397e-04 - mse: 7.7397e-04 - val_loss: 0.1064 - val_mse: 0.1064
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 8.5217e-04 - mse: 8.5217e-04 - val_loss: 0.1277 - val_mse: 0.1277
Epoch 70/150
33/33 [=============

33/33 [==============================] - 0s 4ms/step - loss: 7.6784e-04 - mse: 7.6784e-04 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 124/150
33/33 [==============================] - 0s 4ms/step - loss: 8.0718e-04 - mse: 8.0718e-04 - val_loss: 0.1105 - val_mse: 0.1105
Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0010 - mse: 0.0010 - val_loss: 0.1107 - val_mse: 0.1107
Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0014 - mse: 0.0014 - val_loss: 0.1109 - val_mse: 0.1109
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.1116 - val_mse: 0.1116
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.1146 - val_mse: 0.1146
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0026 - mse: 0.0026 - val_loss: 0.1087 - val_mse: 0.1087
Epoch 130/150
33/33 [==============================] - 0s 4ms/step 

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data
adam_model.evaluate(X_train,y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 6.3620e-04 - mse: 6.3620e-04


[0.0006361976265907288, 0.0006361976265907288]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [26]:
# Evaluate the model on validate data
adam_model.evaluate(X_val,y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1067 - mse: 0.1067


[0.10666779428720474, 0.10666779428720474]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [27]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 4ms/step - loss: 0.1523 - mse: 0.1523


[0.15231077373027802, 0.15231077373027802]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [28]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test,y_test_pred))

5/5 [==============================] - 0s 3ms/step


30668.80290602495

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.